In [9]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import urllib.parse

def urlParser(url):
    url=urllib.parse.unquote(url)
    itemID=url.split("https://www.ebay.com/itm/")[1].split("?")[0]
    return itemID


def get_date(date_str):
    parsed_date = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    return parsed_date.strftime("%A, %d %b 20%y %H:%M:%S UTC")



# Define the XML request body
def getDetails(listingID):    
    xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <GetSingleItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <ItemID>{listingID}</ItemID>
        <IncludeSelector>Details</IncludeSelector>
    </GetSingleItemRequest>"""

    headers = {
        "X-EBAY-API-IAF-TOKEN": "Bearer v^1.1#i^1#I^3#p^3#f^0#r^0#t^H4sIAAAAAAAAAOVZe4wbRxk/3yMkSkMqCkkaULk4lEKitWcfttdL7MTn8905uTs7ti9qTkXu7O6svbn1rrMzez5f1XJ3hagpApWGPqSIKiE8ykskEMQfFQhRtZFalBQq+geoqKJAeZ0UKqQqQqLM2ncX52iT2D5US+w/1n77vX7ffA/PDJhbt2HXsZFjb23yvK/71ByY6/Z42I1gw7q+3e/v6d7e1wUaGDyn5j4217vQ8+c9GJaMspRBuGyZGPXPlAwTSzVixOvYpmRBrGPJhCWEJaJI2djYqMT5gFS2LWIpluHtTw5GvALLhmQUYoGshQQxGKJUc1lnzop4g0EhGOZZENBYmRMVSL9j7KCkiQk0ScTLAU5gAM+wQo7lJY6XAPABnpv09h9CNtYtk7L4gDdac1eqydoNvl7fVYgxsglV4o0mY0PZVCw5mBjP7fE36IouxSFLIHHwtW9xS0X9h6DhoOubwTVuKesoCsLY64/WLVyrVIotO9OC+7VQi1ADQU7goMzJshJcm1AOWXYJkuv74VJ0ldFqrBIyiU6qN4oojYZ8BClk6W2cqkgO9rs/Bx1o6JqO7Ig3MRA7PJFNZLz92XTatqZ1FakuUo4LAVEQRTbsjcrUKxGIoSUbdUVLEV5lJG6Zqu7GC/ePW2QAUYfR6rCwDWGhTCkzZcc04jrTyBdYDh8nTrrrWV9AhxRNd0lRicagv/Z64+AvZ8PV9V+rfICarLAaLyiKgBAvC++cD26tN5cTUXdZYum03/UFybDKlKA9hUjZgApiFBpep4RsXZX4gMbxooYYNRjWGCGsaYwcUIMMqyEEEKJJGhb/T1KDEFuXHYJW0mP1hxq+iDerWGWUtgxdqXpXs9Q6zVIyzOCIt0hIWfL7K5WKr8L7LLvg5wBg/XePjWaVIirR+l/m1W/MzOi1tFAQlcK6RKpl6s0MzTpq3Cx4o7ytpqFNqllkGJSwnLPX+BZdTX0XkHFDpxHIUROdhXHEwgSpbUFT0bSuoLyudhYyjmYoEAJurQsiAEBsC6RhFXRzDJGi1WEwh1Op4dFEW9ho/4Sks1A1dBcQlFhOCoR94QAlhWifaQtsrFxOlkoOgbKBkh22lkKYDQZDbcErO06nFaJsTM1WjsJZu1RqC5o7diUdahKxppD5Tq3UrfX3FmsmMZRJZEfyudSBxHhbaDNIsxEu5lysnZansYOx/TH6jMVzRX95RqgEVWXISA8MDxyYDmI4MpiRx5PcbLKSBo4/kxg+FHdGivDoUEg+aO82Du/GHIcCAMDSwUikrSBlkWKjDmtdQ/LsjIKVZDxdzoweOWIOsdYQZxwphKeS1YlCYDY1JVsFOZGZiSfbAz9W6LRKXxq5azBuc+9W4isA3Vp/T0Da9cLM17pQnr61BTRR6Lh+zfGBINL4MCsiAFmOhTAMgqqsuY+ihpS2x2+H4R2wbBjDBmQMHRNKIUw6M8hwNJnVsAhkJiQImixybJtzudOWea3GMna3b/9baG6tNwvP1YGpEljWfe4/B59ilfwWdEjRJeVrXvffDJMf0+2fr77dp5p9NoKqZRrVVoSbkNHNabphtOxqKwZXhJuQgYpiOSZpxdySaBMSmmNoumG4pwKtGGwQb8ZNExpVoiu4JZO66WYbbkKkDKs1gKqOy2693JQkpZWQrSCfrtZPFVtx1kbUIKwdpLUi1KTJFZdNi+iartR1YEfGiq2Xb96Lmh631q+rq5V4YFoLTS1dXWDFVHvba6TqNlJI3rH1zhoB7uTLu6PPZFYNQQbbRw1NLhVm24LuhrYTD02Sg2uwQxtE0532VybMywIryAIjC+EwIwRknpHDGmA4yIJQUORFURXawry2B0W982fXADQbYoHAcYHQTW85VhEaDqj/61rCf+2VYLSr9rALnp+DBc9Puz0esAfcye4EO9b1TPT23LId64S2bqj5sF4wIXFs5JtC1TLU7e7but786mMj8e2J1OO77stVXzp5oeuWhhvJU58G21buJDf0sBsbLijBR65+6WM3b93ECYBnBZbneAAmwc6rX3vZLb0fTJ076bk9fc+LZy7Pk9svfG39q7s+8QuwaYXJ4+nr6l3wdM2+vDjxAzDwzOlT5yuPVl6tBKOfv/TPRx7/Dpu8vPDy537yIDhx7sr9W+bjVx5S9v3rR0888vy5E4Hhi5tfi/xx9rNv9z38dXTyzTtytz2hfeqVy6mH/U/u3H7f1uf/cubE37594J6fWcd3pO4kf/rhs29s/v22iXt3bb0jP1acnhrd/9b+xdS33lh8cX7y0pd++43Bj86lNpJvfuHsA5PPoVuvPLV4+dkXHrj39ekXpn53uJLdd9fe19f/KvHr08fP/sN5+6XX5k463fnKGf8n5z+uXlJDv/nMj2On7/7Dlu9tW39m3UXPjuPHDmTGL3w3s3fv95/a8qGn/z355ed2Pu3Bjz3zxWO77/rrvlfg+cr9X/nAg3H74pPn/7744V/W1/I/P/8I8yseAAA=",
        "X-EBAY-API-SITE-ID": "0",
        "X-EBAY-API-CALL-NAME": "GetSingleItem",
        "X-EBAY-API-VERSION": "863",
        "X-EBAY-API-REQUEST-ENCODING": "xml"
    }

    # Define the endpoint
    url = 'https://open.api.ebay.com/shopping'

    # Send the request
    response = requests.post(url, headers=headers, data=xml_request)
    print(response.text)
    soup=BeautifulSoup(response.text, 'xml')
    name=soup.find('Title').text
    url="https://www.ebay.com/itm/"+str(listingID)
    start=soup.find('StartTime').text
    print("Name of product:",name)
    print("URL of listing:",url)
    print("ID of listing:",listingID)
    print("Start time of listing:",get_date(start))


getDetails(126361628669)

<?xml version="1.0" encoding="UTF-8"?>

  <GetSingleItemResponse xmlns="urn:ebay:apis:eBLBaseComponents">
   <Timestamp>2024-03-14T13:23:18.656Z</Timestamp>
   <Ack>Success</Ack>
   <Build>1299_CORE_APILW_19146596_R1</Build>
   <Version>1299</Version>
   <Item>
    <BestOfferEnabled>false</BestOfferEnabled>
    <ItemID>126361628669</ItemID>
    <EndTime>2024-04-03T22:33:45.000Z</EndTime>
    <StartTime>2024-03-04T22:33:45.000Z</StartTime>
    <ViewItemURLForNaturalSearch>https://www.ebay.com/itm/Sony-WF-1000XM5-Truly-Wireless-Bluetooth-Noise-Canceling-Headphones-Black-/126361628669</ViewItemURLForNaturalSearch>
    <ListingType>FixedPriceItem</ListingType>
    <Location>Wernersville, Pennsylvania</Location>
    <PictureURL>https://i.ebayimg.com/00/s/MTM0NFgxNDg4/z/dNkAAOSwXCllU6T6/$_12.JPG?set_id=880000500F</PictureURL>
    <PictureURL>https://i.ebayimg.com/00/s/MTYwMFgxMTQw/z/IvkAAOSwTuBlU6T7/$_12.JPG?set_id=880000500F</PictureURL>
    <PictureURL>https://i.ebayimg.com/00/s/NzUxWDExND

In [4]:
import requests
from datetime import datetime

def get_date(date_str):
    parsed_date = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    return parsed_date.strftime("%A, %d %b 20%y %H:%M:%S UTC")

itemID=196160072952
# Define the XML request body
xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
<GetSingleItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
    <ItemID>{itemID}</ItemID>
    <IncludeSelector>Details</IncludeSelector>
</GetSingleItemRequest>"""

headers = {
    "X-EBAY-API-IAF-TOKEN": "Bearer v^1.1#i^1#f^0#p^1#I^3#r^0#t^H4sIAAAAAAAAAOVYbWwURRi+a68FAhQNjWADSdmCQvH29uu+1t6l12trT1uu7RUCyEdnd2ePbfd2rztztldC0vQHCQGMJAWjRKj8QNEoCQpRf2BQDIkYMESIAZGAXyRIAgRCREV3r0e5VgJIL7GJ9+cy77zzzvM8874zs0P1FU+qXN+w/uZU+4SCwT6qr8BupydTk4qLFpYUFpQV2agcB/tg39w+R3/hxSoEEmqSb4UoqWsIlvckVA3xGWOASBkarwOkIF4DCYh4LPKxUFMjz5AUnzR0rIu6SpRHagME8ACJZmjJLXslr9tPm1btTsw23ez3Aq+bkt2yB3g5EXjNfoRSMKIhDDQcIBiK4ZwU66T8bTTLsx6ec5MeH7ucKF8CDaTomulCUkQwA5fPjDVysN4fKkAIGtgMQgQjofpYNBSprVvUVuXKiRXM6hDDAKfQyFZYl2D5EqCm4P2nQRlvPpYSRYgQ4QoOzTAyKB+6A+YR4Gek9tCC4JNNGRng8bJuf16krNeNBMD3x2FZFMkpZ1x5qGEFpx+kqKmG0AFFnG0tMkNEasutv5YUUBVZgUaAqKsJLQs1NxPBGt0AIaQCp6ogrGhx7GxurXUyDENJfh8lOL0cJws+hs5ONBQtK/OomcK6JimWaKh8kY5roIkajtaGydHGdIpqUSMkYwtRrp/7joZe73JrUYdWMYXXaNa6woQpRHmm+eAVGB6NsaEIKQyHI4zuyEhklk0yqUjE6M5MLmbTpwcFiDUYJ3mXq7u7m+xmSd2IuxiKol1Lmxpj4hqYAITpa9X6kL/y4AFOJUNFhOZIpPA4nTSx9Ji5agLQ4kSQ89Mejzer+0hYwdHWfxhyOLtGVkS+KkTy0QInUR7gYVmOY7l8VEgwm6QuCwcUQNqZAEYnxEkViNApmnmWSkBDkXjWLTOsT4ZOyeOXnZxflp2CW/I4aRlCCkJBEP2+/1OhPGyqx6BoQJyXXM9bntcLvT0iEiPh5mRrY0eHVk/r9YzaEfd3RtKL4+7eaKegx4W61p5wJPCw1XBP8mFVMZVpM+fPhwBWredPhAYdYSiNiV5M1JOwWVcVMT2+Fpg1pGZg4HQMqqppGBPJUDIZyc9enTd6/3KbeDTe+Tuj/qPz6Z6skJWy44uVNR6ZAUBSIa0TiBT1hMuqdR2Y1w/LvDqDeky8FfPmOq5YmySH2CrS0JWTzNAl0UsiaUCkpwzztk1GrRtYm94JNfM8w4auqtBYMra8tuo5kUhhIKhwvBV2HhJcAePssKW9lN/vpznKPSZeYuYoXT3etqR8bMWO5x7xWu0a+ZEftGV+dL/9M6rffrDAbqeqqHl0BTWnuHCxo3BKGVIwJBUgk0iJa+a3qwHJTphOAsUomG67tmtrQ7isLrqtcm1b+uvtR2xTct4YBldSM4dfGSYV0pNznhyoWXd7iuhpM6YyHMVSfpplPZx7OVVxt9dBP+EojTJdVU9O2fFVO9e3/fFV6pbiTcJ1auqwk91eZHP0223xby/Ne7Xp1idvEAe+ey9csbs6yuLup86gBdXhGTO3aspm2wSjq/RaJRhYGzi88ccVHx47fO7ArjMXXn7t5PkVg0v5lmVdX+xpb990YsP7x1ZOm3Pl1OkVV/m3yQt/VM9ST948n35T2nu7YuHmiTuXXk0fP73x+j47URUdLCmt+6X3xmOV2yaWvJKo/rhy/47q3z764YTtXNWNOT831XywaffFdavsLxwuOVZ29uhO7/5ntt/a8unlgZPHZ7+4bnrxwXdqj+75ctmlp1Xt2bOHqqqPvE7/1DC/2+fu+uav9JWje+f/3tJ5Yfp5x4LvK56f29hS1H7qLfHdfU3XLhdPrnMESvf/OfvXz29riQ0DvVsODa3l3w8MceT9EQAA",
    "X-EBAY-API-SITE-ID": "0",
    "X-EBAY-API-CALL-NAME": "GetSingleItem",
    "X-EBAY-API-VERSION": "863",
    "X-EBAY-API-REQUEST-ENCODING": "xml"
}

# Define the endpoint
url = 'https://open.api.ebay.com/shopping'

# Send the request
response = requests.post(url, headers=headers, data=xml_request)
print(response.text)
# Print the response
#print(response.text)
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.text, 'xml')


<?xml version="1.0" encoding="UTF-8"?>

  <GetSingleItemResponse xmlns="urn:ebay:apis:eBLBaseComponents">
   <Timestamp>2024-03-14T13:19:12.795Z</Timestamp>
   <Ack>Failure</Ack>
   <Errors>
    <ShortMessage>Invalid token.</ShortMessage>
    <LongMessage>Invalid token. Please specify a valid token as HTTP header.</LongMessage>
    <ErrorCode>1.32</ErrorCode>
    <SeverityCode>Error</SeverityCode>
    <ErrorClassification>RequestError</ErrorClassification>
   </Errors>
   <Build>1299_CORE_APILW_19146596_R1</Build>
   <Version>1299</Version>
  </GetSingleItemResponse>
 


In [3]:
url="https://www.ebay.com/itm/186312828253?_trkparms=amclksrc%3DITM%26aid%3D777008%26algo%3DPERSONAL.TOPIC%26ao%3D1%26asc%3D20230811172246%26meid%3D9eef7ceb962b42968b0b29059af35ced%26pid%3D101783%26rk%3D1%26rkt%3D1%26itm%3D186312828253%26pmt%3D0%26noa%3D1%26pg%3D4375194%26algv%3DFeaturedDealsV2&_trksid=p"
getDetails(urlParser(url))


AttributeError: 'NoneType' object has no attribute 'text'

In [7]:
name=soup.find('Title').text
url="https://www.ebay.com/itm/"+str(itemID)
start=soup.find('StartTime').text
print("Name of product:",name)
print("URL of listing:",url)
print("ID of listing:",itemID)
print("Start time of listing:",get_date(start))

AttributeError: 'NoneType' object has no attribute 'text'